In [104]:
import sys
import os
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize

train_data_size = 1000
test_data_size = 1000
DIC_SIZE = 10000
# nltk.download('stopwords')

In [127]:
# dic = buildVocabulary(os.getcwd()+'/aclImdb/train/')
ytrain = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
ytrain.shape
vocabulary = loadDic(os.getcwd()+'/aclImdb/imdb.vocab')
Xtrain, Xtest, ytrain, ytest = loadData(os.getcwd()+'/aclImdb/', vocabulary)
Xtrain.shape, Xtrain

((2000, 65156), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 6],
        [0, 0, 0, ..., 0, 0, 5],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [126]:
def buildVocabulary(Path):
    tokenizer = RegexpTokenizer(r'\w+') #tokenizer removing puncturations
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    tokens = []
    for folder in os.listdir(Path):
        for filename in os.listdir(Path+folder):
            data = open(Path+folder+'/'+filename, 'r')
            raw = data.read()
            token = tokenizer.tokenize(raw)
            # stop words removal
            filtered_token = [w for w in token if not w in stop_words]
#             filtered_token = [ps.stem(w) for w in token if not w in stop_words]
            # stemming
            for i in range(len(filtered_token)):
                filtered_token[i] = ps.stem(filtered_token[i])
            tokens.extend(filtered_token)
    word_counts = Counter(tokens).most_common(3999)
    vocab = {}
    i = 0
    for word_count in word_counts:
        vocab[word_count[0]] = i
        i += 1
    vocab['UNK'] = i
    return vocab

def loadDic(Path):
    dic = open(Path,'r')
    dic = dic.read().split('\n')
    
    # stop words removal and stemming
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    filtered_dic = list(set([ps.stem(w) for w in dic if not w in stop_words]))
    
    # transfer to dictionary
    
    filtered_dic = {word: i for i, word in enumerate(filtered_dic)}
    filtered_dic['UNK'] = len(filtered_dic)
    return filtered_dic

def loadData(Path, vocabulary):
#     Xtrain, Xtest = [], []
    voc_size = len(vocabulary)
    Xtrain = np.zeros((train_data_size*2,voc_size), dtype=int)
    Xtest = np.zeros((train_data_size*2,voc_size), dtype=int)
    ytrain = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    ytest = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    
    for i, filename in enumerate(os.listdir(Path+'train/pos/')[:train_data_size]):
#         Xtrain.append(transfer(Path+'train/pos/'+filename, vocabulary))
        Xtrain[i] = transfer(Path+'train/pos/'+filename, vocabulary)

    for i, filename in enumerate(os.listdir(Path+'train/neg/')[:train_data_size]):
        Xtrain[i] = transfer(Path+'train/neg/'+filename, vocabulary)
#         Xtrain.append(transfer(Path+'train/neg/'+filename, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'test/pos/')[:test_data_size]):
        Xtest[i] = transfer(Path+'test/pos/'+filename, vocabulary)
#         Xtest.append(transfer(Path+'test/pos/'+filename, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'test/neg/')[:test_data_size]):
        Xtest[i] = transfer(Path+'test/neg/'+filename, vocabulary)
#         Xtest.append(transfer(Path+'test/neg/'+filename, vocabulary))

#     Xtrain, Xtest = np.array(Xtrain), np.array(Xtest)
    return Xtrain, Xtest, ytrain, ytest


def transfer(fileDj, vocabulary):
    data = open(fileDj,'r')
    BOWDj = np.zeros((len(vocabulary),), dtype = int)
    
    tokenizer = RegexpTokenizer(r'\w+') #tokenizer removing puncturations
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    
    raw = data.read()
    tokens = tokenizer.tokenize(raw)
    # stop words removal
    filtered_token = [ps.stem(w) for w in tokens if not w in stop_words]
#     # stemming
#     for i in range(len(filtered_token)):
#         filtered_token[i] = ps.stem(filtered_token[i])
    # transfer to BOW
    for token in filtered_token:
        if token in vocabulary:
            BOWDj[vocabulary[token]] += 1
        else:
            BOWDj[vocabulary['UNK']] += 1
    
    return BOWDj

